In [1]:

%tensorflow_version 2.x

UsageError: Line magic function `%tensorflow_version` not found.


In [0]:
%load_ext tensorboard

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

ModuleNotFoundError: No module named 'tensorflow'

In [0]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

# Variables, gradients

In [0]:
x = tf.Variable(initial_value=[1.0, 2.0, 3.0, 4.0])

In [0]:
with tf.GradientTape() as g:
  y = tf.reduce_sum(tf.square(x))

In [0]:
g.gradient(y, x)

# CIFAR-10
See https://www.cs.toronto.edu/~kriz/cifar.html

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = tf.cast(x_train, tf.float32) / 255
x_test = tf.cast(x_test, tf.float32) / 255
y_train = tf.squeeze(tf.cast(y_train, tf.int32))
y_test = tf.squeeze(tf.cast(y_test, tf.int32))

In [0]:
labels = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

In [0]:
img_number =  5102#@param {type:"integer"}

plt.imshow(np.squeeze(x_train[img_number]))
labels[y_train[img_number]]

# Low-level tensorflow without keras

## 2-layer convolutional network / ResNet

In [0]:
class CifarCNN(tf.Module):
    def __init__(self):
        initializer = tf.keras.initializers.glorot_normal()
        self._filter1 = tf.Variable(initializer([3, 3, 3, 8]))
        self._filter2 = tf.Variable(initializer([3, 3, 8, 5]))
        self._filter_project = tf.Variable(initializer([1, 1, 8, 5]))
        self._fcn_weights = tf.Variable(
            initializer([16 * 16 * 5, 10])
    )

    @tf.function(input_signature=[tf.TensorSpec([None, 32, 32, 3])])
    def Infer(self, input_batch):
        # input_batch is (batch_size x 32 x 32 x 3)
        net = tf.nn.conv2d(input_batch, self._filter1, strides=[1,1], padding="SAME")
        net = tf.nn.relu(net)
        net = tf.nn.max_pool2d(net, ksize=[2, 2], strides=[2, 2], padding="SAME")
        residuals = tf.nn.relu(tf.nn.conv2d(
            net, self._filter2, strides=[1,1], padding="SAME"))
        net = residuals + tf.nn.conv2d(
            net, self._filter_project, strides=[1,1], padding="SAME")
        net = tf.reshape(net, [tf.shape(net)[0], -1])
        net = tf.matmul(net, self._fcn_weights)
    return net

    def GetLoss(self, x_batch, y_batch):
        losses = tf.nn.sparse_softmax_cross_entropy_with_logits(
              tf.squeeze(y_batch),
              self.Infer(x_batch))
        return tf.reduce_mean(losses)

    def Accuracy(self, x, y_true):
        logits = self.Infer(x)
        return tf.reduce_mean(
            tf.cast(tf.math.equal(
                tf.cast(tf.math.argmax(logits, 1), y_true.dtype), 
                y_true), tf.float32))

    @tf.function
    def MakeStep(self, x_batch, y_batch, optimizer):
        with tf.GradientTape() as g:
            loss = self.GetLoss(x_batch, y_batch)
        g = g.gradient(loss, self.trainable_variables)

        # SGD step
        optimizer.apply_gradients(zip(g, self.trainable_variables))

In [0]:
cnn = CifarCNN()

In [0]:
cnn.Accuracy(x_test, y_test)

In [0]:
batch_size = 10
epochs = 5

optimizer = tf.keras.optimizers.Adam()
_ = cnn.Infer(x_train[0:10])
for j in range(epochs):
    print(f"Epoch {j}: test accuracy {cnn.Accuracy(x_test, y_test)}")
    for i in range(5000):
    cnn.MakeStep(
        x_train[(i*batch_size):((i+1)*batch_size)],
        y_train[(i*batch_size):((i+1)*batch_size)],
        optimizer)


# Keras sequential API

See https://www.tensorflow.org/api_docs/python/tf/keras/Sequential

In [0]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(filters=8, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.MaxPool2D(strides=2))
model.add(tf.keras.layers.Conv2D(filters=5, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [0]:
logdir="logs/sequential_fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [0]:
model.fit(x_train, 
          y_train,
          batch_size=10,
          epochs=3,
          verbose=1,
          validation_data=(
              x_test, 
              y_test),
          callbacks=[tensorboard_callback])

In [0]:
%tensorboard --logdir logs

# Keras functional API

See https://www.tensorflow.org/guide/keras/functional

In [0]:
inputs = tf.keras.layers.Input(shape=[32, 32, 3])
layer1 = tf.keras.layers.Conv2D(filters=8, kernel_size=3, activation='relu', padding="same")(inputs)
layer2 = tf.keras.layers.MaxPool2D(strides=2)(layer1)
layer3_resid = tf.keras.layers.Conv2D(filters=5, kernel_size=3, activation='relu', padding="same")(layer2)
layer2_proj = tf.keras.layers.Conv2D(filters=5, kernel_size=1, padding="same")(layer2)
layer3 = layer2_proj + layer3_resid
layer4 = tf.keras.layers.Flatten()(layer3)
layer5 = tf.keras.layers.Dense(10, activation='softmax')(layer4)

In [0]:
model = tf.keras.Model(inputs=inputs, outputs=layer5)

In [0]:
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, 
          y_train,
          batch_size=10,
          epochs=5,
          verbose=1,
          validation_data=(
              x_test, 
              y_test))

# Subclassing Keras layer

See https://www.tensorflow.org/guide/keras/custom_layers_and_models

In [0]:
class ResidualLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(ResidualLayer, self).__init__()
        self._conv = tf.keras.layers.Conv2D(filters=5, kernel_size=3, activation='relu', padding='same')
        self._project = tf.keras.layers.Conv2D(filters=5, kernel_size=1, padding='same')

    def call(self, inputs):
    return self._project(inputs) + self._conv(inputs)

In [0]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(filters=8, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.MaxPool2D(strides=2))
model.add(ResidualLayer())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, 
          y_train,
          batch_size=10,
          epochs=5,
          verbose=1,
          validation_data=(
              x_test, 
              y_test))

# Saving models

See https://www.tensorflow.org/guide/keras/save_and_serialize

## Keras model

In [0]:
model.save("models/sequential")

In [0]:
restored_sequential_model = tf.keras.models.load_model("models/sequential")

In [0]:
type(restored_sequential_model)

## Custom model

In [0]:
tf.saved_model.save(
    cnn, 
    "models/custom_cnn",
    signatures=cnn.Infer)

In [0]:
restored_custom_cnn = tf.saved_model.load("models/custom_cnn")

In [0]:
type(restored_custom_cnn)

In [0]:
restored_custom_cnn.Infer(x_train)